# 🎯 Lecture 8: Hardware-Aware NAS - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/08_neural_architecture_search_2/demo.ipynb)

## What You'll Learn
- Why hardware-aware NAS matters
- Latency and FLOPs as constraints
- Latency lookup tables
- Multi-objective optimization

In [ ]:
!pip install torch matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time

torch.manual_seed(42)
print('Ready for Hardware-Aware NAS!')

## Part 1: The Hardware Efficiency Gap

**Problem**: Theoretical FLOPs ≠ Actual Latency

Different operations have different hardware efficiency.

In [ ]:
def measure_latency(op, x, num_runs=100):
    """Measure actual latency of an operation."""
    # Warmup
    for _ in range(10):
        _ = op(x)
    
    # Measure
    start = time.time()
    for _ in range(num_runs):
        _ = op(x)
    return (time.time() - start) / num_runs * 1000  # ms

def count_flops(op, x):
    """Estimate FLOPs for common operations."""
    if isinstance(op, nn.Conv2d):
        h, w = x.shape[2], x.shape[3]
        out_h = h - op.kernel_size[0] + 1 + 2 * op.padding[0]
        out_w = w - op.kernel_size[1] + 1 + 2 * op.padding[1]
        return 2 * op.in_channels * op.out_channels * op.kernel_size[0] * op.kernel_size[1] * out_h * out_w
    return 0

# Compare different operations
x = torch.randn(1, 64, 56, 56)

operations = {
    'Conv 3x3': nn.Conv2d(64, 64, 3, padding=1),
    'Conv 5x5': nn.Conv2d(64, 64, 5, padding=2),
    'Conv 7x7': nn.Conv2d(64, 64, 7, padding=3),
    'Depthwise 3x3': nn.Conv2d(64, 64, 3, padding=1, groups=64),
    'Depthwise 5x5': nn.Conv2d(64, 64, 5, padding=2, groups=64),
    'Conv 1x1': nn.Conv2d(64, 64, 1),
}

print('📊 FLOPS vs LATENCY COMPARISON')
print('=' * 70)
print(f'{"Operation":<20} {"FLOPs (M)":<15} {"Latency (ms)":<15} {"FLOPS/ms":<15}')
print('-' * 70)

results = []
for name, op in operations.items():
    flops = count_flops(op, x) / 1e6
    latency = measure_latency(op, x)
    efficiency = flops / latency if latency > 0 else 0
    results.append({'name': name, 'flops': flops, 'latency': latency, 'efficiency': efficiency})
    print(f'{name:<20} {flops:<15.2f} {latency:<15.3f} {efficiency:<15.2f}')

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

names = [r['name'] for r in results]
flops = [r['flops'] for r in results]
latencies = [r['latency'] for r in results]
efficiencies = [r['efficiency'] for r in results]

axes[0].barh(names, flops, color='#3b82f6')
axes[0].set_xlabel('FLOPs (M)')
axes[0].set_title('Theoretical Compute')

axes[1].barh(names, latencies, color='#ef4444')
axes[1].set_xlabel('Latency (ms)')
axes[1].set_title('Actual Latency')

axes[2].barh(names, efficiencies, color='#22c55e')
axes[2].set_xlabel('FLOPS/ms')
axes[2].set_title('Hardware Efficiency')

plt.tight_layout()
plt.show()

print('\n💡 Key Insight: Same FLOPs ≠ Same Latency!')

## Part 2: Latency Lookup Tables

In [ ]:
class LatencyTable:
    """
    Pre-measured latency lookup table for different operations.
    In practice, this is measured on the target hardware.
    """
    def __init__(self):
        # Latency in milliseconds for different (op, channels, resolution) combinations
        # These are example values - real tables are measured on target hardware
        self.table = {
            # (operation, in_channels, out_channels, resolution)
            ('conv3x3', 16, 16, 32): 0.05,
            ('conv3x3', 16, 32, 32): 0.08,
            ('conv3x3', 32, 32, 16): 0.06,
            ('conv3x3', 32, 64, 16): 0.10,
            ('conv3x3', 64, 64, 8): 0.08,
            
            ('conv5x5', 16, 16, 32): 0.12,
            ('conv5x5', 16, 32, 32): 0.18,
            ('conv5x5', 32, 32, 16): 0.14,
            ('conv5x5', 32, 64, 16): 0.22,
            ('conv5x5', 64, 64, 8): 0.18,
            
            ('sep_conv3x3', 16, 16, 32): 0.03,
            ('sep_conv3x3', 16, 32, 32): 0.04,
            ('sep_conv3x3', 32, 32, 16): 0.03,
            ('sep_conv3x3', 32, 64, 16): 0.05,
            ('sep_conv3x3', 64, 64, 8): 0.04,
            
            ('skip', 16, 16, 32): 0.001,
            ('skip', 32, 32, 16): 0.001,
            ('skip', 64, 64, 8): 0.001,
        }
    
    def get_latency(self, op_name, in_ch, out_ch, resolution):
        key = (op_name, in_ch, out_ch, resolution)
        return self.table.get(key, 0.1)  # Default latency

latency_table = LatencyTable()

print('📊 LATENCY LOOKUP TABLE')
print('=' * 70)
print(f'{"Operation":<15} {"Channels":<15} {"Resolution":<15} {"Latency (ms)":<15}')
print('-' * 70)

for key, latency in list(latency_table.table.items())[:10]:
    op, in_ch, out_ch, res = key
    print(f'{op:<15} {in_ch}→{out_ch:<10} {res}×{res:<10} {latency:<15.3f}')

print('\n💡 Lookup tables enable fast latency estimation during search!')

## Part 3: Hardware-Aware Mixed Operation

In [ ]:
class HardwareAwareMixedOp(nn.Module):
    """
    Mixed operation with latency awareness.
    Includes latency cost in the architecture optimization.
    """
    def __init__(self, in_channels, out_channels, resolution, latency_table):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.resolution = resolution
        self.latency_table = latency_table
        
        # Operations
        self.ops = nn.ModuleDict({
            'conv3x3': nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
                nn.BatchNorm2d(out_channels), nn.ReLU()),
            'conv5x5': nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 5, padding=2, bias=False),
                nn.BatchNorm2d(out_channels), nn.ReLU()),
            'sep_conv3x3': nn.Sequential(
                nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels, bias=False),
                nn.Conv2d(in_channels, out_channels, 1, bias=False),
                nn.BatchNorm2d(out_channels), nn.ReLU()),
            'skip': nn.Identity() if in_channels == out_channels else 
                   nn.Conv2d(in_channels, out_channels, 1),
        })
        
        self.op_names = list(self.ops.keys())
        self.alpha = nn.Parameter(torch.zeros(len(self.op_names)))
    
    def forward(self, x):
        weights = F.softmax(self.alpha, dim=0)
        out = sum(w * self.ops[name](x) for w, name in zip(weights, self.op_names))
        return out
    
    def get_latency(self):
        """Get expected latency based on current architecture weights."""
        weights = F.softmax(self.alpha, dim=0)
        latency = 0
        for w, name in zip(weights, self.op_names):
            op_latency = self.latency_table.get_latency(
                name, self.in_channels, self.out_channels, self.resolution)
            latency += w * op_latency
        return latency

# Demo
hw_mixed_op = HardwareAwareMixedOp(16, 16, 32, latency_table)

print('📊 HARDWARE-AWARE MIXED OPERATION')
print('=' * 50)
print('\nOperation latencies:')
for name in hw_mixed_op.op_names:
    lat = latency_table.get_latency(name, 16, 16, 32)
    print(f'  {name:<15}: {lat:.3f} ms')

print(f'\nExpected latency: {hw_mixed_op.get_latency().item():.3f} ms')

## Part 4: Latency-Constrained NAS

In [ ]:
class LatencyAwareSupernet(nn.Module):
    """Supernet with latency constraints."""
    def __init__(self, latency_table, num_classes=10):
        super().__init__()
        self.latency_table = latency_table
        
        self.stem = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.BatchNorm2d(16), nn.ReLU())
        
        # Searchable layers with different resolutions
        self.layer1 = HardwareAwareMixedOp(16, 16, 32, latency_table)
        self.layer2 = HardwareAwareMixedOp(16, 32, 16, latency_table)
        self.layer3 = HardwareAwareMixedOp(32, 64, 8, latency_table)
        
        self.pool1 = nn.MaxPool2d(2)
        self.pool2 = nn.MaxPool2d(2)
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.pool1(x)
        x = self.layer2(x)
        x = self.pool2(x)
        x = self.layer3(x)
        x = self.gap(x).flatten(1)
        return self.fc(x)
    
    def get_total_latency(self):
        """Get total expected latency of the network."""
        return (self.layer1.get_latency() + 
                self.layer2.get_latency() + 
                self.layer3.get_latency())
    
    def get_arch_params(self):
        return [self.layer1.alpha, self.layer2.alpha, self.layer3.alpha]
    
    def get_weight_params(self):
        return [p for n, p in self.named_parameters() if 'alpha' not in n]

def train_latency_aware(supernet, X_train, y_train, X_val, y_val, 
                        latency_target=0.3, epochs=30):
    """
    Train with latency constraint in the loss.
    
    Loss = CE_loss + lambda * max(0, latency - target)
    """
    weight_opt = torch.optim.SGD(supernet.get_weight_params(), lr=0.01, momentum=0.9)
    arch_opt = torch.optim.Adam(supernet.get_arch_params(), lr=0.001)
    
    criterion = nn.CrossEntropyLoss()
    lambda_latency = 10.0  # Weight for latency penalty
    
    history = {'loss': [], 'latency': [], 'acc': []}
    
    for epoch in range(epochs):
        supernet.train()
        
        # Update weights
        weight_opt.zero_grad()
        loss = criterion(supernet(X_train), y_train)
        loss.backward()
        weight_opt.step()
        
        # Update architecture with latency penalty
        arch_opt.zero_grad()
        ce_loss = criterion(supernet(X_val), y_val)
        latency = supernet.get_total_latency()
        latency_penalty = F.relu(latency - latency_target)
        total_loss = ce_loss + lambda_latency * latency_penalty
        total_loss.backward()
        arch_opt.step()
        
        # Track metrics
        with torch.no_grad():
            acc = (supernet(X_val).argmax(1) == y_val).float().mean().item() * 100
        
        history['loss'].append(ce_loss.item())
        history['latency'].append(latency.item())
        history['acc'].append(acc)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}: Loss={ce_loss.item():.3f}, '
                  f'Latency={latency.item():.3f}ms, Acc={acc:.1f}%')
    
    return history

# Create data
X_train = torch.randn(200, 3, 32, 32)
y_train = torch.randint(0, 10, (200,))
X_val = torch.randn(100, 3, 32, 32)
y_val = torch.randint(0, 10, (100,))

print('🔄 TRAINING LATENCY-AWARE SUPERNET')
print('=' * 50)
print('Target latency: 0.15 ms')

supernet = LatencyAwareSupernet(latency_table)
history = train_latency_aware(supernet, X_train, y_train, X_val, y_val, 
                              latency_target=0.15, epochs=40)

In [ ]:
# Visualize training
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['loss'], color='#3b82f6')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('CE Loss')
axes[0].set_title('Classification Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['latency'], color='#ef4444')
axes[1].axhline(y=0.15, color='green', linestyle='--', label='Target')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Latency (ms)')
axes[1].set_title('Network Latency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['acc'], color='#22c55e')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Accuracy (%)')
axes[2].set_title('Validation Accuracy')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show final architecture
print('\n🏆 DISCOVERED ARCHITECTURE')
print('=' * 50)
for i, layer in enumerate([supernet.layer1, supernet.layer2, supernet.layer3]):
    weights = F.softmax(layer.alpha, dim=0).detach()
    best_idx = weights.argmax().item()
    print(f'Layer {i+1}: {layer.op_names[best_idx]} ({weights[best_idx].item():.1%})')

print(f'\nFinal latency: {supernet.get_total_latency().item():.3f} ms')
print(f'Target was: 0.15 ms')

## Part 5: Pareto-Optimal Architecture Search

In [ ]:
def pareto_search(latency_targets, X_train, y_train, X_val, y_val):
    """
    Search for Pareto-optimal architectures.
    Different latency targets give different accuracy-efficiency trade-offs.
    """
    pareto_front = []
    
    for target in latency_targets:
        print(f'\nSearching with latency target: {target} ms')
        
        supernet = LatencyAwareSupernet(latency_table)
        
        # Quick training
        weight_opt = torch.optim.SGD(supernet.get_weight_params(), lr=0.01, momentum=0.9)
        arch_opt = torch.optim.Adam(supernet.get_arch_params(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        
        for epoch in range(30):
            supernet.train()
            
            weight_opt.zero_grad()
            criterion(supernet(X_train), y_train).backward()
            weight_opt.step()
            
            arch_opt.zero_grad()
            ce_loss = criterion(supernet(X_val), y_val)
            latency = supernet.get_total_latency()
            (ce_loss + 10 * F.relu(latency - target)).backward()
            arch_opt.step()
        
        # Evaluate
        with torch.no_grad():
            acc = (supernet(X_val).argmax(1) == y_val).float().mean().item() * 100
            final_latency = supernet.get_total_latency().item()
        
        # Get architecture
        arch = []
        for layer in [supernet.layer1, supernet.layer2, supernet.layer3]:
            weights = F.softmax(layer.alpha, dim=0).detach()
            arch.append(layer.op_names[weights.argmax().item()])
        
        pareto_front.append({
            'target': target,
            'latency': final_latency,
            'accuracy': acc,
            'architecture': arch
        })
        
        print(f'  Result: {acc:.1f}% accuracy, {final_latency:.3f} ms latency')
        print(f'  Architecture: {arch}')
    
    return pareto_front

print('🔍 PARETO-OPTIMAL ARCHITECTURE SEARCH')
print('=' * 50)

pareto_results = pareto_search(
    latency_targets=[0.05, 0.10, 0.15, 0.20, 0.30],
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val
)

In [ ]:
# Visualize Pareto front
fig, ax = plt.subplots(figsize=(10, 6))

latencies = [r['latency'] for r in pareto_results]
accuracies = [r['accuracy'] for r in pareto_results]

ax.scatter(latencies, accuracies, s=200, c='#3b82f6', zorder=5)
ax.plot(latencies, accuracies, 'b--', alpha=0.5, label='Pareto Front')

for r in pareto_results:
    ax.annotate(f'{r["architecture"][0][:4]}...', 
                (r['latency'], r['accuracy']),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

ax.set_xlabel('Latency (ms)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('📊 Pareto Front: Accuracy vs Latency', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()

# Shade dominated region
ax.fill_between([0, max(latencies)*1.1], [min(accuracies)]*2, [0]*2, 
                alpha=0.1, color='red', label='Dominated')

plt.tight_layout()
plt.show()

print('\n📊 PARETO-OPTIMAL ARCHITECTURES')
print('=' * 70)
print(f'{"Target":<10} {"Latency":<12} {"Accuracy":<12} {"Architecture":<30}')
print('-' * 70)
for r in pareto_results:
    print(f'{r["target"]:<10.2f} {r["latency"]:<12.3f} {r["accuracy"]:<12.1f} {str(r["architecture"]):<30}')

print('\n💡 Choose architecture based on your latency budget!')

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. FLOPs ≠ Latency - hardware efficiency varies by operation')
print('\n2. Latency lookup tables enable fast hardware-aware search')
print('\n3. Add latency penalty to loss for constrained optimization')
print('\n4. Different targets → different accuracy-latency trade-offs')
print('\n5. Pareto front shows all optimal trade-off points')
print('\n6. Target-specific architectures beat one-size-fits-all')
print('\n' + '=' * 60)
print('\n📚 Next: Knowledge Distillation!')